<a href="https://colab.research.google.com/github/NaughtyFishies/mines_PE_alum/blob/main/20_normalize_and_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
%run /content/drive/MyDrive/mines-alum/00_bootstrap_env.ipynb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.1/572.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 10.4 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Authentication failed! Check your access token.
Directory structure ready:
raw        -> /content/drive/MyDrive/mines-alum/raw
interim    -> /content/drive/MyDrive/mines-alum/interim
processed  -> /content/drive/MyDrive/mines-alum/processed
profiles   -> /content/drive/MyDrive/mines-alum/profiles
index      -> /content/drive/MyDrive/mines-alum/index
logs       -> /content/drive/MyDrive/mines-alum/logs
config     -> /content/drive/MyDrive/mines-alum/config
[2025-10-27 16:19:57] Wrote sample log to Drive
[2025-10-27 16:19:58] Wrote environment report to Drive


In [18]:
import os
import dropbox
import json
import sys

In [8]:
sys.path.append('/content/drive/MyDrive/mines-alum')
from dropbox_utils import init_dbx, test_dbx

dbx = init_dbx()

In [9]:
PROJECT_ROOT = "/content/drive/MyDrive/mines-alum"
RAW_DIR = f"{PROJECT_ROOT}/raw"
INTERIM_DIR = f"{PROJECT_ROOT}/interim"

In [12]:
def detect_file_type(entry):
    if entry.name.lower().endswith(('.jpg')):
        file_type = 'jpg'
    elif entry.name.lower().endswith('.pdf'):
        file_type = 'pdf'
    elif entry.name.lower().endswith(('.txt', '.csv')):
        file_type = 'txt'
    elif entry.name.lower().endswith(('.docx')):
        file_type = 'docx'
    elif entry.name.lower().endswith(('.pptx')):
        file_type = 'pptx'
    elif entry.name.lower().endswith(('.xlsx')):
        file_type = 'xlsx'
    else:
        file_type = 'unknown'

    return {
      "name": entry.name,
      "dropbox_path": entry.path_lower,
      "mime_type": file_type,
      "size": entry.size
      }

In [13]:
def enumerate_and_classify_files(dbx, folder):
    all_entries = []
    result = dbx.files_list_folder(folder, recursive=True)

    entries = result.entries
    while result.has_more:
        result = dbx.files_list_folder_continue(result.cursor)
        entries.extend(result.entries)

    for entry in entries:
        if isinstance(entry, dropbox.files.FileMetadata):
            info = detect_file_type(entry)
            all_entries.append(info)

    return all_entries

In [19]:
DROPBOX_FOLDER = "/Hall of Fame Committee"
classified_files = enumerate_and_classify_files(dbx, DROPBOX_FOLDER)

os.makedirs(INTERIM_DIR, exist_ok=True)
with open(f"{INTERIM_DIR}/file_types.json", "w") as f:
    json.dump(classified_files, f, indent=2)